<a href="https://colab.research.google.com/github/saransh2320/Twiiter-Customer-Support-/blob/master/Customer_Support.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import json

In [2]:
!pip install pycld2

     |████████████████████████████████| 41.4MB 92kB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp36-cp36m-linux_x86_64.whl size=9833491 sha256=7c237677fa0f1895595afa1488d0dfbb93d5cc31770de4a00fac07c5ee07842a
  Stored in directory: /root/.cache/pip/wheels/c6/8f/e9/08a1a8932a490175bd140206cd86a3dbcfc70498100de11079
Successfully built pycld2


In [3]:
!pip install -q kaggle

In [4]:
kaggle_api_key = '{"username":"hsaransh","key":"d751f4719ec73f12f570241faafd19c9"}'
kaggle_api_key = json.loads(kaggle_api_key)

In [5]:
!mkdir -p ~/.kaggle

In [6]:
with open('kaggle.json', 'w') as json_file:
    json.dump(kaggle_api_key, json_file)

In [7]:
!cp kaggle.json ~/.kaggle/

In [8]:
!chmod 600 /root/.kaggle/kaggle.json

In [9]:
!kaggle datasets download -d thoughtvector/customer-support-on-twitter

 94% 158M/169M [00:01<00:00, 125MB/s]
100% 169M/169M [00:01<00:00, 103MB/s]


In [10]:
!unzip '/content/customer-support-on-twitter.zip'

Archive:  /content/customer-support-on-twitter.zip
  inflating: sample.csv              
  inflating: twcs/twcs.csv           


In [11]:
import numpy as np
import pandas as pd
import string

In [12]:
tweets = pd.read_csv('/content/twcs/twcs.csv')

In [13]:
tweets.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [14]:
tweets.loc[lambda tf: tf['tweet_id'] == 81]

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
73,81,ChipotleTweets,False,Mon Oct 30 15:00:02 +0000 2017,Just stop by in costume 10/31 3pm-close. Offic...,"80,82,83,84,85,86,87,88,89,90,91,92,93,94,95,9...",NaN


In [15]:
tweets.loc[lambda tf: tf['tweet_id'] == 409]

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
301,409,McDonalds,False,Tue Oct 31 16:02:50 +0000 2017,So what’ll it be this #Halloween: a trick or a...,"410,411,412,413,414,415,416,417,418,419,420,42...",NaN


In [16]:
tweets.shape

(2811774, 7)

In [17]:
# First_inbound have all the tweets in_response_to_tweet_id is null (i.e. starting of converstion) and tweets.inbound == true (i.e. started by customer)
first_inbound = tweets[pd.isnull(tweets.in_response_to_tweet_id) & tweets.inbound == True]

inbounds_and_outbounds = pd.merge(first_inbound,tweets,left_on='tweet_id',right_on='in_response_to_tweet_id')

inbounds_and_outbounds = inbounds_and_outbounds[inbounds_and_outbounds.inbound_y ^ True]

In [18]:
inbounds_and_outbounds.head()

,tweet_id_x,author_id_x,inbound_x,created_at_x,text_x,response_tweet_id_x,in_response_to_tweet_id_x,tweet_id_y,author_id_y,inbound_y,created_at_y,text_y,response_tweet_id_y,in_response_to_tweet_id_y
0,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN,6,sprintcare,False,Tue Oct 31 21:46:24 +0000 2017,@115712 Can you please send us a private messa...,"5,7",8.0
1,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN,9,sprintcare,False,Tue Oct 31 21:46:14 +0000 2017,@115712 I would love the chance to review the ...,NaN,8.0
2,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN,10,sprintcare,False,Tue Oct 31 21:45:59 +0000 2017,@115712 Hello! We never like our customers to ...,NaN,8.0
3,18,115713,True,Tue Oct 31 19:56:01 +0000 2017,@115714 y’all lie about your “great” connectio...,17,NaN,17,sprintcare,False,Tue Oct 31 19:59:13 +0000 2017,@115713 H there! We'd definitely like to work ...,16,18.0
4,20,115715,True,Tue Oct 31 22:03:34 +0000 2017,"@115714 whenever I contact customer support, t...",19,NaN,19,sprintcare,False,Tue Oct 31 22:10:10 +0000 2017,@115715 Please send me a private message so th...,NaN,20.0


In [19]:
inbounds_and_outbounds.shape

(794299, 14)

In [20]:
# query_and_response = inbounds_and_outbounds.drop(['tweet_id_x' ,'author_id_x' ,'inbound_x' ,'created_at_x' ,'response_tweet_id_x','in_response_to_tweet_id_x','tweet_id_y','author_id_y','inbound_y','created_at_y','response_tweet_id_y','in_response_to_tweet_id_y'],axis=1)

In [36]:
query_and_response = inbounds_and_outbounds[["text_x","text_y"]]
query_and_response.head()

,text_x,text_y
0,@sprintcare is the worst customer service,@115712 Can you please send us a private messa...
1,@sprintcare is the worst customer service,@115712 I would love the chance to review the ...
2,@sprintcare is the worst customer service,@115712 Hello! We never like our customers to ...
3,@115714 y’all lie about your “great” connectio...,@115713 H there! We'd definitely like to work ...
4,"@115714 whenever I contact customer support, t...",@115715 Please send me a private message so th...


In [37]:
query_and_response.shape

(794299, 2)

In [38]:
query_and_response['text_x'].replace(to_replace="@[A-Za-z0-9]+", value="", regex=True,inplace=True)

query_and_response['text_y'].replace(to_replace="@[A-Za-z0-9]+", value="", regex=True,inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [39]:
query_and_response.head()

,text_x,text_y
0,is the worst customer service,"Can you please send us a private message, so ..."
1,is the worst customer service,I would love the chance to review the account...
2,is the worst customer service,Hello! We never like our customers to feel li...
3,y’all lie about your “great” connection. 5 ba...,H there! We'd definitely like to work with yo...
4,"whenever I contact customer support, they tel...",Please send me a private message so that I ca...


In [40]:
# Testing pycld2 Library
import pycld2 as cld2
k = (cld2.detect("who is here")[2])[0][1]

In [41]:
def is_en(txt):
    try:
        return ((cld2.detect(txt)[2])[0][1])=='en'
    except:
        return False

query_and_response = query_and_response[query_and_response['text_x'].apply(is_en)]
query_and_response = query_and_response[query_and_response['text_y'].apply(is_en)]

In [42]:
query_and_response.shape

(754225, 2)

In [43]:
query_and_response["text_x"] = query_and_response["text_x"].str.lower()
query_and_response["text_y"] = query_and_response["text_y"].str.lower()


In [44]:
# PUNCT_TO_REMOVE = string.punctuation

def remove_punctuation(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])# It will discard all punctuations
    return text_nopunct

query_and_response["text_x"] = query_and_response["text_x"].apply(lambda text: remove_punctuation(text))
query_and_response["text_y"] = query_and_response["text_y"].apply(lambda text: remove_punctuation(text))


In [45]:
query_and_response.head(20)

,text_x,text_y
0,is the worst customer service,can you please send us a private message so t...
1,is the worst customer service,i would love the chance to review the account...
2,is the worst customer service,hello we never like our customers to feel lik...
3,y’all lie about your “great” connection 5 bar...,h there wed definitely like to work with you ...
4,whenever i contact customer support they tell...,please send me a private message so that i ca...
5,actually thats a broken link you sent me and i...,the information pertaining to the account ass...
6,yo spectrum your customer service reps are sup...,hello my apologies for any frustrations or in...
7,my picture on spectrum pretty much every day w...,i apologize for the inconvenience i will be g...
8,somebody from please help meeeeee 😩😩😩😩 im hav...,help has arrived we are sorry to see that you...
9,my friend is without internet we need to play...,have your friend message us\nacm


In [54]:
freqX = pd.Series(''.join(query_and_response['text_x']).split()).value_counts()[:10]
freqY = pd.Series(''.join(query_and_response['text_y']).split()).value_counts()[:10]

In [55]:
print(freqX)
print("-----------")
print(freqY)

to     448257
the    387885
i      379725
my     302624
a      281177
and    248641
is     207291
for    198060
on     181021
you    168152
dtype: int64
-----------
to      589910
you     553615
the     434419
your    350594
we      293641
us      276569
for     276531
can     257925
a       251868
this    243893
dtype: int64


In [57]:
freqX = list(freqX.index)
freqY = list(freqY.index)

In [71]:
xx = "whenever i contact customer support they my"
axx = " ".join(xx for xx in xx.split() if xx not in freqX)
axx = "";
for xx in xx.split():
  if(xx not in freqX):
    axx += xx
    axx += ' '
axx

'whenever contact customer support they '

In [72]:
query_and_response["text_x"] = query_and_response["text_x"].apply(lambda x : " ".join(x for x in x.split() if x not in freqX))
query_and_response["text_y"] = query_and_response["text_y"].apply(lambda x : " ".join(x for x in x.split() if x not in freqY))

In [73]:
query_and_response.head(20)

,text_x,text_y
0,worst customer service,please send private message so that i gain fur...
1,worst customer service,i would love chance review account and provide...
2,worst customer service,hello never like our customers feel like they ...
3,y’all lie about your “great” connection 5 bars...,h there wed definitely like work with on how l...
4,whenever contact customer support they tell me...,please send me private message so that i send ...
5,actually thats broken link sent me incorrect i...,information pertaining account assumption is c...
6,yo spectrum your customer service reps are sup...,hello my apologies any frustrations or inconve...
7,picture spectrum pretty much every day why sho...,i apologize inconvenience i will be glad assis...
8,somebody from please help meeeeee 😩😩😩😩 im havi...,help has arrived are sorry see that are having...
9,friend without internet we need play videogame...,have friend message acm


In [75]:
import nltk
import re

In [76]:
def Tokenize(txt):
  tokens = re.split('\W+',txt)
  return tokens

In [78]:
query_and_response['text_x'] = query_and_response['text_x'].apply(lambda x: Tokenize(x))
query_and_response['text_y'] = query_and_response['text_y'].apply(lambda x: Tokenize(x))


In [79]:
query_and_response.head()

,text_x,text_y
0,"[worst, customer, service]","[please, send, private, message, so, that, i, ..."
1,"[worst, customer, service]","[i, would, love, chance, review, account, and,..."
2,"[worst, customer, service]","[hello, never, like, our, customers, feel, lik..."
3,"[y, all, lie, about, your, great, connection, ...","[h, there, wed, definitely, like, work, with, ..."
4,"[whenever, contact, customer, support, they, t...","[please, send, me, private, message, so, that,..."


In [87]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [93]:
from nltk.stem import WordNetLemmatizer

In [101]:
wnl = WordNetLemmatizer()

def lemmatizing(txt):
  txt = [wnl.lemmatize(word) for word in txt]
  return txt

In [102]:
lemmatizing(['Was', 'has', 'cycling','cycle','what'])

['Was', 'ha', 'cycling', 'cycle', 'what']